<a href="https://colab.research.google.com/github/sobti/TSAI/blob/master/S9/QuizDnn_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-03-21 17:29:04.105064


In [3]:
#Mounting Google Colab and importing generic Packages
import torchvision
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from albumentations.pytorch import ToTensor
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from google.colab import drive
import os
import sys
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import torchvision
import torch
import albumentations
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose)

drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/Colab Notebooks\resnet')
os.chdir('/content/gdrive/My Drive/Colab Notebooks/resnet')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
#importing self-created packages:
import train_test_model_resnet18 as t                                           #import train - test module .This is class to train and evaluate the model
import load as ld                                                               #dataloader
import mod_summ as m                                                            #Getting model summary package
import Basic_Block as bb                                                        #Getting Classes for Resnet and Basic Block
import album as al                                                              #Getting albumentation module
import override_album as oa                                                     # overriding the albumentation class
import Test_Module as tt                                                        # Test Block


In [5]:
# Test BLOCK
net=tt.Net().to('cuda')
net.forward(torch.randn(1,3,32,32).to('cuda')).to('cuda')
m.summ_print(net,(3,32,32))# Model summary.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
           Dropout-4           [-1, 16, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           5,472
              ReLU-6           [-1, 32, 32, 32]               0
       BatchNorm2d-7           [-1, 32, 32, 32]              64
           Dropout-8           [-1, 32, 32, 32]               0
         MaxPool2d-9           [-1, 51, 16, 16]               0
           Conv2d-10           [-1, 64, 16, 16]          29,376
             ReLU-11           [-1, 64, 16, 16]               0
      BatchNorm2d-12           [-1, 64, 16, 16]             128
          Dropout-13           [-1, 64, 16, 16]               0
           Conv2d-14          [-1, 128,

/content/gdrive/My Drive/Colab Notebooks/resnet/Test_Module.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x13)


In [0]:
train_transforms,test_transforms=al.album_transform(albumentations)             #Albumentation class 

In [7]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=None)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=None)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
# Initialize the dataset, pass the augmentation pipeline as an argument to init function
train_ds = oa.PyTorchImageDataset(image=trainset, augmentations=train_transforms)
test_ds =  oa.PyTorchImageDataset(image=testset, augmentations=test_transforms)

In [0]:
batch_size=64
trainloader,testloader=ld.load_test_train(batch_size,train_ds,test_ds,torchvision,torch)

In [10]:
train_losses = []
test_losses = []
train_acc = []
test_acc = []
optimizer = optim.SGD(net.parameters(),lr=0.043,weight_decay=0.0019,momentum=0.90)
criterion=nn.CrossEntropyLoss()
scheduler=StepLR(optimizer,step_size=5,gamma=0.1)
#optimizer=optim.Adam(model.parameters(), lr=0.045, betas=(0.9, 0.999), eps=1e-08, amsgrad=False)
EPOCHS = 10
device='cuda'
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    t.train(net, device, trainloader, optimizer, epoch,train_losses,train_acc,tqdm,criterion)
    t.test(net, device, testloader,test_losses,test_acc,tqdm,criterion,torch)
    scheduler.step()


  0%|          | 0/782 [00:00<?, ?it/s]

EPOCH: 0


/content/gdrive/My Drive/Colab Notebooks/resnet/Test_Module.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x13)
Loss=1.6571389436721802 Batch_id=781 Accuracy=41.84: 100%|██████████| 782/782 [00:31<00:00, 24.57it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.0221, Accuracy: 5148/10000 (51.48%)

EPOCH: 1


Loss=1.337122917175293 Batch_id=781 Accuracy=56.76: 100%|██████████| 782/782 [00:31<00:00, 24.49it/s]
  0%|          | 0/782 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0089, Accuracy: 8078/10000 (80.78%)

EPOCH: 8


Loss=0.7310627698898315 Batch_id=781 Accuracy=76.50: 100%|██████████| 782/782 [00:32<00:00, 24.37it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.0089, Accuracy: 8041/10000 (80.41%)

EPOCH: 9


Loss=0.9151359796524048 Batch_id=781 Accuracy=77.00: 100%|██████████| 782/782 [00:31<00:00, 24.71it/s]



Test set: Average loss: 0.0086, Accuracy: 8086/10000 (80.86%)

